# Audiobooks predicting a customer will Buy again.

Data Description: Data from an Audiobook app.
Each customer in the database has made a purchase at least once.
Variables in dataset:
Customer ID, Book length in mins_avg (average of all purchases), Book length in minutes_sum (sum of all purchases), Price Paid_avg (average of all purchases), Price paid_sum (sum of all purchases), Review (a Boolean variable), Review (out of 10), Total minutes listened, Completion (from 0 to 1), Support requests (number), and Last visited minus purchase date (in days).

    Project: Creating a machine Learning algorithm based on our data to predict if a customer will buy again from the Audiobook company.
    And, use the results to make more effective marketing decisions

In [2]:
#Importing Packages

In [21]:
import numpy as np
from sklearn import preprocessing
import tensorflow as tf

In [7]:
raw_csv_data = np.loadtxt ('Audiobooks_data.csv', delimiter = ',')

unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

In [8]:
# Balancing the Dataset

In [9]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
            
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis = 0)
targets_equal_priors = np.delete (targets_all, indices_to_remove, axis = 0)

In [10]:
# Standardizing the Inputs

In [11]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

In [12]:
# Shuffling the Data

In [15]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

## Spliting Data into TRAIN, VALIDATION, and TEST

In [18]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

# Verifying we have balanced data sets
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1794.0 3579 0.501257334450964
228.0 447 0.5100671140939598
215.0 448 0.4799107142857143


## Saving datasets in *.npz format 

In [20]:
np.savez('Audiobooks_data_train', inputs= train_inputs, targets =train_targets)
np.savez('Audiobooks_data_validation', inputs= validation_inputs, targets =validation_targets)
np.savez('Audiobooks_data_test', inputs= test_inputs, targets =test_targets)

In [22]:
#Data

In [24]:
npz = np.load('Audiobooks_data_train.npz')
train_inputs, train_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

## Defining Model outline

Outline, optimizer, loss, early stopping and training

In [29]:
input_size = 10 
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax'),
                            ])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 100

max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,
         train_targets,
         batch_size = batch_size,
         epochs = max_epochs,
         callbacks=[early_stopping],
         validation_data=(validation_inputs, validation_targets),
         verbose=2)

Train on 3579 samples, validate on 447 samples
Epoch 1/100
3579/3579 - 0s - loss: 0.5555 - accuracy: 0.7941 - val_loss: 0.4399 - val_accuracy: 0.8926
Epoch 2/100
3579/3579 - 0s - loss: 0.3841 - accuracy: 0.8737 - val_loss: 0.3187 - val_accuracy: 0.9060
Epoch 3/100
3579/3579 - 0s - loss: 0.3215 - accuracy: 0.8793 - val_loss: 0.2772 - val_accuracy: 0.9128
Epoch 4/100
3579/3579 - 0s - loss: 0.2998 - accuracy: 0.8863 - val_loss: 0.2606 - val_accuracy: 0.9150
Epoch 5/100
3579/3579 - 0s - loss: 0.2852 - accuracy: 0.8891 - val_loss: 0.2449 - val_accuracy: 0.9172
Epoch 6/100
3579/3579 - 0s - loss: 0.2770 - accuracy: 0.8935 - val_loss: 0.2355 - val_accuracy: 0.9217
Epoch 7/100
3579/3579 - 0s - loss: 0.2708 - accuracy: 0.8941 - val_loss: 0.2296 - val_accuracy: 0.9217
Epoch 8/100
3579/3579 - 0s - loss: 0.2648 - accuracy: 0.8980 - val_loss: 0.2283 - val_accuracy: 0.9262
Epoch 9/100
3579/3579 - 0s - loss: 0.2594 - accuracy: 0.9003 - val_loss: 0.2193 - val_accuracy: 0.9329
Epoch 10/100
3579/3579 - 0

## Testing the Model

In [32]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

448/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [33]:
print('\nTest: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test: 0.25. Test accuracy: 91.07%


Our Model have accuracy of 91% which is we can correctly predict whether a customer will buy again or not 9 out of 10 times.